# TLC Data Prep & Cleaning and Feature Engineering

In [1]:
import h3
import geopandas as gpd
import pandas as pdb

[Data Source](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
> Download data: https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip.    
> Unzip the file and store under uberProj/data/.

In [2]:
# Read shapefile
zones_gdf = gpd.read_file("../data/taxi_zones/taxi_zones.shp")
zones_gdf = zones_gdf.to_crs(epsg=4326) # convert to WGS84 from Web Mercator

In [4]:
# Convert to hexes based on geo
def get_hexes(geo, res=8):
    try:
        return h3.geo_to_cells(geo, res)
    except Exception as e:
        print("Error in get_hexes:", e)
        return set()

zones_gdf['h3_list'] = zones_gdf.geometry.apply(lambda x: get_hexes(x))
zones_gdf['distribution_factor'] = zones_gdf['h3_list'].apply(
    lambda x: 1.0 / len(x) if len(x) > 0 else 0
)

zones_gdf.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,h3_list,distribution_factor
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.695, -74.18449 40.6951,...","[882a1071edfffff, 882a1071e7fffff, 882a1071e5f...",0.100000
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...","[882a10395dfffff, 882a103955fffff, 882a103951f...",0.055556
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870...","[882a100103fffff, 882a10011dfffff, 882a100119f...",0.250000
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725...",[],0.000000
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562...","[882a106017fffff, 882a1060e1fffff, 882a1060e5f...",0.125000


In [8]:
# Explode to create lookup table
zone_lookup = zones_gdf[['LocationID', 'h3_list', 'distribution_factor']].explode('h3_list')
zone_lookup = zone_lookup.rename(columns={'h3_list': 'h3_index'}).dropna()

print(f"{zone_lookup['h3_index'].nunique()} unique H3 cells (full coverage)")

# Save
zone_lookup.to_parquet('../data/zone_h3_lookup_polyfill.parquet', index=False)
print(f"Saved zone lookups to ../data/zone_h3_lookup_polyfill.parquet")


1070 unique H3 cells (full coverage)
Saved zone lookups to ../data/zone_h3_lookup_polyfill.parquet


## Zone Lookup

In [9]:
import duckdb
import pandas as pd

# Load the polyfill lookup
zone_lookup = pd.read_parquet('../data/zone_h3_lookup_polyfill.parquet')

print(f"Zone lookup shape: {zone_lookup.shape}")
print(f"Unique H3 cells: {zone_lookup['h3_index'].nunique()}")
print("\nSample of zone lookup:")
print(zone_lookup.head())

# Connect to DuckDB and register the lookup
con = duckdb.connect()
con.register('zone_lookup', zone_lookup)

# Build file list
files = []
for year in range(2022, 2025):
    for month in range(1, 13):
        files.append(
            f"'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'"
        )

# 2025: only Jan-Oct
for month in range(1, 11):
    files.append(
        f"'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-{month:02d}.parquet'"
    )

files_list = f"[{', '.join(files)}]"

# 4. QUERY for distribution (using duckdb enhanced speed by ~50x)
query = f"""
WITH trips_with_zones AS (
    SELECT
        t.tpep_pickup_datetime,
        z.h3_index,
        z.distribution_factor
    FROM read_parquet({files_list}) t
    JOIN zone_lookup z ON t.PULocationID = z.LocationID
    WHERE t.tpep_pickup_datetime >= '2022-01-01' 
      AND t.tpep_pickup_datetime < '2025-11-01'
),
hourly_agg AS (
    SELECT
        h3_index,
        date_trunc('hour', tpep_pickup_datetime) AS hour_bin,
        -- IMPORTANT: Use SUM(distribution_factor) to distribute traffic
        SUM(distribution_factor) AS traffic_count
    FROM trips_with_zones
    GROUP BY h3_index, hour_bin
)
SELECT
    h3_index,
    hour_bin + INTERVAL '1 hour' AS match_hour,
    traffic_count
FROM hourly_agg
ORDER BY h3_index, match_hour;
"""

print("\nRunning query ...")
traffic_polyfill = con.execute(query).fetchdf()

print(f"\n--- RESULTS ---")
print(f"Total records: {len(traffic_polyfill):,}")
print(f"Date range: {traffic_polyfill['match_hour'].min()} to {traffic_polyfill['match_hour'].max()}")
print(f"Unique H3 cells: {traffic_polyfill['h3_index'].nunique()}")
print(f"\nTraffic statistics:")
print(traffic_polyfill['traffic_count'].describe())

Zone lookup shape: (1070, 3)
Unique H3 cells: 1070

Sample of zone lookup:
   LocationID         h3_index  distribution_factor
0           1  882a1071edfffff                  0.1
1           1  882a1071e7fffff                  0.1
2           1  882a1071e5fffff                  0.1
3           1  882a1071adfffff                  0.1
4           1  882a1071e3fffff                  0.1

Running query ...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


--- RESULTS ---
Total records: 11,619,229
Date range: 2022-01-01 01:00:00 to 2025-11-01 00:00:00
Unique H3 cells: 1070

Traffic statistics:
count    1.161923e+07
mean     1.135127e+01
std      3.777377e+01
min      3.448276e-02
25%      2.000000e-01
50%      5.000000e-01
75%      2.800000e+00
max      1.239000e+03
Name: traffic_count, dtype: float64


In [10]:
# Compare with collision data
collision_cells = set(pd.read_csv('../data/h3_full_panel_res8.csv')['h3_index'].unique())
traffic_cells = set(traffic_polyfill['h3_index'].unique())

overlap = collision_cells & traffic_cells
missing = collision_cells - traffic_cells

print(f"\n--- COVERAGE ANALYSIS ---")
print(f"Collision cells: {len(collision_cells)}")
print(f"Traffic cells: {len(traffic_cells)}")
print(f"Overlapping cells: {len(overlap)} ({len(overlap)/len(collision_cells)*100:.1f}%)")
print(f"Collision cells with ZERO traffic: {len(missing)} ({len(missing)/len(collision_cells)*100:.1f}%)")


--- COVERAGE ANALYSIS ---
Collision cells: 1135
Traffic cells: 1070
Overlapping cells: 997 (87.8%)
Collision cells with ZERO traffic: 138 (12.2%)


In [11]:
# Filter to valid date range (clean any bad dates)
traffic_clean = traffic_polyfill[
    (traffic_polyfill['match_hour'].dt.year >= 2022) & 
    (traffic_polyfill['match_hour'].dt.year <= 2025)
].copy()

print(f"\nAfter cleaning: {len(traffic_clean):,} records")

# Save the traffic data
traffic_clean.to_parquet('../data/traffic_h3_2022_2025_polyfill.parquet', index=False)
print("✓ Saved to ../data/traffic_h3_2022_2025_polyfill.parquet")


After cleaning: 11,619,229 records
✓ Saved to ../data/traffic_h3_2022_2025_polyfill.parquet


In [12]:
# Missing cells & Crashes in missing cells

collision_df = pd.read_csv('../data/h3_full_panel_res8.csv')
traffic_df = pd.read_parquet('../data/traffic_h3_2022_2025_polyfill.parquet')

collision_cells = set(collision_df['h3_index'].unique())
traffic_cells = set(traffic_df['h3_index'].unique())
missing_cells = collision_cells - traffic_cells

print(f"Missing cells: {list(missing_cells)[:10]}")

# Check how many crashes are in these missing cells
missing_collisions = collision_df[collision_df['h3_index'].isin(missing_cells)]
print(f"\nCrashes in missing cells: {missing_collisions['accidents_count'].sum():,}")
print(f"% of total crashes: {missing_collisions['accidents_count'].sum() / collision_df['accidents_count'].sum() * 100:.2f}%")

Missing cells: ['882a10750dfffff', '882a1001dbfffff', '882a10776bfffff', '882a10019dfffff', '882a106267fffff', '882a1072dbfffff', '882a1076d3fffff', '882a107213fffff', '882a10742dfffff', '882a103827fffff']

Crashes in missing cells: 10,275
% of total crashes: 3.02%


## Validation & Integration Tests

In [ ]:
# 1. Load all datasets
print("Loading datasets...")
weather_df = pd.read_csv('../data/nyc_weather_hourly.csv')

print(f"\u2713 Traffic data: {len(traffic_df):,} rows, {traffic_df['h3_index'].nunique()} H3 cells")
print(f"\u2713 Collision panel: {len(collision_df):,} rows, {collision_df['h3_index'].nunique()} H3 cells")
print(f"\u2713 Weather data: {len(weather_df):,} rows")

Loading datasets...
✓ Traffic data: 11,619,229 rows, 1070 H3 cells
✓ Collision panel: 38,871,480 rows, 1135 H3 cells
✓ Weather data: 34,248 rows


In [15]:
# 2. Validate traffic data integrity
print("\n=== TRAFFIC DATA VALIDATION ===")

# Date range
traffic_df['match_hour'] = pd.to_datetime(traffic_df['match_hour'])
print(f"Date range: {traffic_df['match_hour'].min()} to {traffic_df['match_hour'].max()}")

# Check for nulls
nulls = traffic_df.isnull().sum()
print(f"\nNull values:\n{nulls}")
assert nulls.sum() == 0, "Traffic data has null values!"

# Traffic statistics
print(f"\nTraffic count statistics:")
print(traffic_df['traffic_count'].describe())

# Check for negative or zero traffic
invalid = traffic_df[traffic_df['traffic_count'] <= 0]
print(f"\nRows with zero/negative traffic: {len(invalid):,} ({len(invalid)/len(traffic_df)*100:.2f}%)")

print("\u2713 Traffic data integrity: PASSED")


=== TRAFFIC DATA VALIDATION ===
Date range: 2022-01-01 01:00:00 to 2025-11-01 00:00:00

Null values:
h3_index         0
match_hour       0
traffic_count    0
dtype: int64

Traffic count statistics:
count    1.161923e+07
mean     1.135127e+01
std      3.777377e+01
min      3.448276e-02
25%      2.000000e-01
50%      5.000000e-01
75%      2.800000e+00
max      1.239000e+03
Name: traffic_count, dtype: float64

Rows with zero/negative traffic: 0 (0.00%)
✓ Traffic data integrity: PASSED


In [16]:
# 3. Coverage analysis: collision cells vs traffic cells
print("\n=== COVERAGE ANALYSIS ===")

collision_cells = set(collision_df['h3_index'].unique())
traffic_cells = set(traffic_df['h3_index'].unique())

overlap = collision_cells & traffic_cells
missing_traffic = collision_cells - traffic_cells
extra_traffic = traffic_cells - collision_cells

print(f"Collision H3 cells: {len(collision_cells):,}")
print(f"Traffic H3 cells: {len(traffic_cells):,}")
print(f"Overlapping cells: {len(overlap):,} ({len(overlap)/len(collision_cells)*100:.1f}%)")
print(f"Collision cells WITHOUT traffic: {len(missing_traffic):,} ({len(missing_traffic)/len(collision_cells)*100:.1f}%)")
print(f"Traffic cells NOT in collisions: {len(extra_traffic):,}")

# Check how many crashes occur in cells with no traffic data
if len(missing_traffic) > 0:
    missing_collisions = collision_df[collision_df['h3_index'].isin(missing_traffic)]
    total_crashes = collision_df['accidents_count'].sum()
    missing_crashes = missing_collisions['accidents_count'].sum()
    print(f"\nCrashes in cells WITHOUT traffic: {missing_crashes:,} / {total_crashes:,} ({missing_crashes/total_crashes*100:.2f}%)")
    print(f"Sample missing cells: {list(missing_traffic)[:5]}")

print("\u2713 Coverage analysis: COMPLETE")


=== COVERAGE ANALYSIS ===
Collision H3 cells: 1,135
Traffic H3 cells: 1,070
Overlapping cells: 997 (87.8%)
Collision cells WITHOUT traffic: 138 (12.2%)
Traffic cells NOT in collisions: 73

Crashes in cells WITHOUT traffic: 10,275 / 339,749 (3.02%)
Sample missing cells: ['882a10750dfffff', '882a1001dbfffff', '882a10776bfffff', '882a10019dfffff', '882a106267fffff']
✓ Coverage analysis: COMPLETE


In [17]:
# 4. Temporal alignment test
print("\n=== TEMPORAL ALIGNMENT ===")

# Normalize datetime columns
collision_df['date'] = pd.to_datetime(collision_df['date'])
collision_df['datetime'] = collision_df['date'] + pd.to_timedelta(collision_df['hour'], unit='h')

weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df['datetime'] = weather_df['date'] + pd.to_timedelta(weather_df['hour'], unit='h')

# Date ranges
print(f"Collision range: {collision_df['datetime'].min()} to {collision_df['datetime'].max()}")
print(f"Traffic range: {traffic_df['match_hour'].min()} to {traffic_df['match_hour'].max()}")
print(f"Weather range: {weather_df['datetime'].min()} to {weather_df['datetime'].max()}")

# Find overlap window
collision_start, collision_end = collision_df['datetime'].min(), collision_df['datetime'].max()
traffic_start, traffic_end = traffic_df['match_hour'].min(), traffic_df['match_hour'].max()
weather_start, weather_end = weather_df['datetime'].min(), weather_df['datetime'].max()

overlap_start = max(collision_start, traffic_start, weather_start)
overlap_end = min(collision_end, traffic_end, weather_end)

print(f"\nCommon overlap: {overlap_start} to {overlap_end}")
print(f"Overlap duration: {(overlap_end - overlap_start).days} days")

assert overlap_start < overlap_end, "No temporal overlap between datasets!"
print("\u2713 Temporal alignment: PASSED")


=== TEMPORAL ALIGNMENT ===
Collision range: 2022-01-01 00:00:00 to 2025-11-27 23:00:00
Traffic range: 2022-01-01 01:00:00 to 2025-11-01 00:00:00
Weather range: 2022-01-01 00:00:00 to 2025-11-27 23:00:00

Common overlap: 2022-01-01 01:00:00 to 2025-11-01 00:00:00
Overlap duration: 1399 days
✓ Temporal alignment: PASSED


In [18]:
# 5. Sample merge test (spot-check integration)
print("\n=== SAMPLE MERGE TEST ===")

# Take a sample H3 cell from the overlap
sample_cell = list(overlap)[0]
sample_start = overlap_start
sample_end = sample_start + pd.Timedelta(days=7)

print(f"Sample cell: {sample_cell}")
print(f"Sample week: {sample_start.date()} to {sample_end.date()}")

# Extract sample data
collision_sample = collision_df[
    (collision_df['h3_index'] == sample_cell) &
    (collision_df['datetime'] >= sample_start) &
    (collision_df['datetime'] < sample_end)
]

traffic_sample = traffic_df[
    (traffic_df['h3_index'] == sample_cell) &
    (traffic_df['match_hour'] >= sample_start) &
    (traffic_df['match_hour'] < sample_end)
]

weather_sample = weather_df[
    (weather_df['datetime'] >= sample_start) &
    (weather_df['datetime'] < sample_end)
]

print(f"\nSample data sizes:")
print(f"  Collision rows: {len(collision_sample)}")
print(f"  Traffic rows: {len(traffic_sample)}")
print(f"  Weather rows: {len(weather_sample)}")

# Attempt merge on (h3, datetime)
merged = collision_sample.merge(
    traffic_sample, 
    left_on=['h3_index', 'datetime'], 
    right_on=['h3_index', 'match_hour'],
    how='left'
)

print(f"\nMerged rows: {len(merged)}")
print(f"Traffic matched: {merged['traffic_count'].notna().sum()} / {len(merged)}")

if len(merged) > 0:
    print(f"\nSample merged data:")
    print(merged[['h3_index', 'datetime', 'accidents_count', 'traffic_count', 'rain_flag']].head())

print("\u2713 Sample merge: PASSED")


=== SAMPLE MERGE TEST ===
Sample cell: 882a10002bfffff
Sample week: 2022-01-01 to 2022-01-08

Sample data sizes:
  Collision rows: 168
  Traffic rows: 1
  Weather rows: 168

Merged rows: 168
Traffic matched: 1 / 168

Sample merged data:
          h3_index            datetime  accidents_count  traffic_count  \
0  882a10002bfffff 2022-01-01 01:00:00                0            NaN   
1  882a10002bfffff 2022-01-01 02:00:00                0            NaN   
2  882a10002bfffff 2022-01-01 03:00:00                0            NaN   
3  882a10002bfffff 2022-01-01 04:00:00                0            NaN   
4  882a10002bfffff 2022-01-01 05:00:00                0            NaN   

   rain_flag  
0          0  
1          0  
2          0  
3          0  
4          1  
✓ Sample merge: PASSED


In [20]:
# 6. Integration readiness summary
print("\n" + "="*60)
print("INTEGRATION READINESS SUMMARY")
print("="*60)

checks = {
    "Traffic data integrity": True,
    "Coverage overlap": len(overlap) / len(collision_cells) > 0.8,
    "Temporal alignment": (overlap_end - overlap_start).days > 365,
    "Merge compatibility": len(merged) > 0
}

for check, passed in checks.items():
    status = "\u2713 PASS" if passed else "\u2717 FAIL"
    print(f"{status} {check}")

if all(checks.values()):
    print("\n\u2705 ALL CHECKS PASSED - Ready for integration!")
    print(f"Ready for causal inference with real traffic data!")
else:
    print("\n\u26a0\ufe0f SOME CHECKS FAILED - Review before integration")

print("="*60)


INTEGRATION READINESS SUMMARY
✓ PASS Traffic data integrity
✓ PASS Coverage overlap
✓ PASS Temporal alignment
✓ PASS Merge compatibility

✅ ALL CHECKS PASSED - Ready for integration!
Ready for causal inference with real traffic data!
